In [0]:
!pip install xlrd==1.2.0
!pip install lxml
!pip install xgboost

Requirement already satisfied: xlrd==1.2.0 in /databricks/python3/lib/python3.8/site-packages (1.2.0)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: lxml in /databricks/python3/lib/python3.8/site-packages (4.8.0)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: xgboost in /databricks/python3/lib/python3.8/site-packages (1.5.2)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.8/site-packages (from xgboost) (1.20.1)
Requirement already satisfied: scipy in /databricks/python3/lib/python3.8/site-packages (from xgboost) (1.6.2)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
from datetime import datetime as dt
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
import pickle
import xgboost
import os
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

random_state = 33
run_date = '2022_02_25'

In [0]:
def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

In [0]:
final_data = spark.read.table('analytical_data_int_ext_04_03_v1')  #enter table name
final_data = final_data.toPandas()

In [0]:
final_data.head()

In [0]:
#Filtering model data for columns required for modelling
cols_to_keep = ['Plant_ID', 'Vendor', 'updated_VS_ID', 'Purchase_Order', 'PO_Item', 'PO_Create_Date', 'Material_No.', 'Material_Description', 
                'vendor_percentage', 'vendor_material_cnt_per_plant', 'dom_or_int_int', 'sku_type_single', 'distance', 'POqty_to_AvgDelQty', 
                'nestle_managed_freight_yes', 'mode_of_transport_road', 'Material_group_type_R', 'single_or_multisource_single', 
                'no.of.complaint_3_months', 'no.of.complaint_12_months', 'no.of.complaint_3_months_major', 
                'no.of.complaint_3_months_minor', 'no.of.complaint_12_months_major', 'no.of.complaint_12_months_minor', 
                'no.of.complaint_3_months_open', 'no.of.complaint_3_months_open_major', 'no.of.complaint_3_months_open_minor', 
                'no.of.Complaint_type_Documentation_l12m', 'no.of.Complaint_type_Packaging_l12m', 'no.of.Complaint_type_Transportation_l12m', 
                'no.of.Complaint_type_Ingredient_l12m', 'no.of.Complaint_type_Consumer_l12m', 'no.of.Complaint_type_Foreign_Material_l12m', 
                'no.of.Complaint_type_Indigenous_Material_l12m', 'Risk_Level_High', 'Risk_Level_Low', 'Risk_Level_Medium', 'Rating_A', 
                'Rating_B', 'Rating_C', 'Rating_D', 'Rating_D_-_FND', 'Rating_F', 'Rating_Inactive', 'Rating_Out_of_Scope', 
                'Delivery_freq_Daily', 'Delivery_freq_Half_Yearly', 'Delivery_freq_Monthly', 'Delivery_freq_Quarterly', 'Delivery_freq_Weekly', 
                'Delivery_freq_Yearly', 'Complaint_type_Documentation_l12m_distinct', 'Complaint_type_Packaging_l12m_distinct', 
                'Complaint_type_Transportation_l12m_distinct', 'Complaint_type_Ingredient_l12m_distinct', 
                'Complaint_type_Consumer_l12m_distinct', 'Complaint_type_Foreign_Material_l12m_distinct', 
                'Complaint_type_Indigenous_Material_l12m_distinct', 'bus_day_count']

final_data_v1 = final_data[cols_to_keep]

In [0]:
#Creating list of material ids to model
mat_ids_to_model = final_data_v1['Material_No.'].unique().tolist()

In [0]:
#Creating a list of continuous variables
cont_vars = ['vendor_percentage', 'vendor_material_cnt_per_plant', 'distance', 'POqty_to_AvgDelQty', 'no.of.complaint_3_months', 
             'no.of.complaint_12_months', 'no.of.complaint_3_months_major', 'no.of.complaint_3_months_minor','no.of.complaint_12_months_major', 
             'no.of.complaint_12_months_minor', 'no.of.complaint_3_months_open', 'no.of.complaint_3_months_open_major', 
             'no.of.complaint_3_months_open_minor', 'no.of.Complaint_type_Documentation_l12m', 'no.of.Complaint_type_Packaging_l12m', 
             'no.of.Complaint_type_Transportation_l12m', 'no.of.Complaint_type_Ingredient_l12m', 'no.of.Complaint_type_Consumer_l12m',
             'no.of.Complaint_type_Foreign_Material_l12m', 'no.of.Complaint_type_Indigenous_Material_l12m', 'bus_day_count']

In [0]:
#Function for outlier removal and scaling (if required)
def model_data_creation(data, mat_id):
  model_data = data[data['Material_No.']==mat_id]
  model_data = model_data[model_data['bus_day_count']>0]
  cont_var_df = model_data[[col for col in model_data.columns.tolist() if col in cont_vars]]
  col_outlier_list = []
  for col in cont_var_df.columns.tolist():
    if col in ['vendor_percentage', 'vendor_material_cnt_per_plant', 'distance', 'POqty_to_AvgDelQty', 'bus_day_count']:
      col_mean = cont_var_df[col].mean()
      col_sd = cont_var_df[col].std()
      upper_outlier_data = cont_var_df[cont_var_df[col]>col_mean+2*col_sd][[col]]
      upper_outlier_data['outlier_type'] = 'upper'
      lower_outlier_data = cont_var_df[cont_var_df[col]<col_mean-2*col_sd][[col]]
      lower_outlier_data['outlier_type'] = 'lower'
      outlier_data = pd.concat([upper_outlier_data, lower_outlier_data], ignore_index=True)
      outlier_data.rename(columns={col:'outlier_value'}, inplace=True)
      outlier_data['col_name'] = col
      outlier_data['col_mean'] = col_mean
      outlier_data['col_sd'] = col_sd
      col_outlier_list.append(outlier_data)
      cont_var_df.loc[cont_var_df[col]>col_mean+2*col_sd, col] = col_mean+2*col_sd
      cont_var_df.loc[cont_var_df[col]<col_mean-2*col_sd, col] = col_mean-2*col_sd
  cont_var_df = cont_var_df.reset_index()
  cont_var_df.drop(columns=['index'], inplace=True)
  cat_var_df = model_data[[col for col in model_data.columns.tolist() if col not in cont_vars]].reset_index()
  cat_var_df.drop(columns=['index'], inplace=True)
  model_data_v1 = pd.concat([cont_var_df, cat_var_df], axis=1)
  outlier_mat_df = pd.concat(col_outlier_list, ignore_index=True)
  
  return model_data_v1, outlier_mat_df

In [0]:
model_list = ['xgb', 'rf', 'avg']

In [0]:
#For loop for running different models
for model_name in model_list:
  
  #Creating list for storing outputs
  summary_df_list = []
  model_data_pred_df_list = []
  mat_id_not_modelled = []
  outlier_data_list = []
  model_data_transformed = []
  
  print(model_name)
  #For loop for modelling each material and storing the respective outputs
  for mat_id in mat_ids_to_model:

    #Creating a dictionary to store model performance summary
    model_var_dict = dict()

    #Applying data transformation function to get outlier data and model data without outliers
    model_data, outlier_mat_df = model_data_creation(final_data_v1, mat_id)

    print(mat_id)

    #Applying try catch to handle exceptions
    try:
      if model_name != 'avg':

        #Creating independent feature dataset
        model_independent_data = model_data.drop(columns=['Plant_ID', 'Vendor', 'updated_VS_ID', 'Purchase_Order', 'PO_Item', 'PO_Create_Date', 
                                                          'Material_No.', 'Material_Description', 'bus_day_count'])

        #Creating target variable
        model_target = model_data['bus_day_count']

        #Creating train test split
        X_train, X_test, Y_train, Y_test = train_test_split(model_independent_data, model_target, test_size = 0.3, random_state = random_state)

        #Creating a param grid for gridsearchcv and model instance
        if model_name == 'rf':
          param_grid = {
          'max_depth': [80, 90, 100],
          'max_features': ['sqrt', 'auto'],
          'min_samples_leaf': [2, 3, 4],
          'min_samples_split': [2, 4, 6],
          'n_estimators': [50, 100, 200, 300, 400, 500]
          }
          model = RandomForestRegressor(random_state=random_state)
        elif model_name == 'xgb':
          param_grid = {
            'booster': ['gbtree'],
            'n_estimators': [300],
            'eta': [0.01]
          }
          model = xgboost.XGBRegressor()

        #Creating grid search instance
        grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                                cv = 3, n_jobs = -1, verbose = 2, refit = True)
        grid_result = grid_search.fit(X_train, Y_train)

        #Using best estimator from grid search to predict Y_train and calculating train error metrics
        Y_train_pred = grid_result.predict(X_train)
        train_mape = 100*mape(Y_train, Y_train_pred)
        train_smape = smape(Y_train, Y_train_pred)
        train_rmse = mse(Y_train, Y_train_pred, squared=False)

        #Using best estimator from grid search to predict Y_train and calculating test error metrics
        Y_test_pred = grid_result.predict(X_test)
        test_mape = 100*mape(Y_test, Y_test_pred)
        test_smape = smape(Y_test, Y_test_pred)
        test_rmse = mse(Y_test, Y_test_pred, squared=False)

        #Using best estimator from grid search to score
        y_pred = grid_result.predict(model_independent_data)
        model_data_pred = model_data.copy()
        model_data_pred['bus_day_count_pred'] = y_pred

        #Saving the best estimator
        model_to_save = grid_result.best_estimator_
        root_directory = '/dbfs/FileStore/models/'
        directory = run_date+'/'+str(mat_id)+'/'
        path = os.path.join(root_directory, directory)
        os.makedirs(path, exist_ok=True)
        file_name = path + model_name + '_reg.pkl'
        pickle.dump(model_to_save, open(file_name, "wb"))

        #Creating list to save descriptive statistics of independent variables
        mean_list = [model_independent_data[feat].describe()[1] for feat in model_independent_data.columns.tolist()]
        SD_list = [model_independent_data[feat].describe()[2] for feat in model_independent_data.columns.tolist()]
        min_list = [model_independent_data[feat].describe()[3] for feat in model_independent_data.columns.tolist()]
        max_list = [model_independent_data[feat].describe()[7] for feat in model_independent_data.columns.tolist()]
        list_5_pct = [model_independent_data[feat].quantile(q=0.05) for feat in model_independent_data.columns.tolist()]
        list_10_pct = [model_independent_data[feat].quantile(q=0.1) for feat in model_independent_data.columns.tolist()]
        list_90_pct = [model_independent_data[feat].quantile(q=0.90) for feat in model_independent_data.columns.tolist()]
        list_95_pct = [model_independent_data[feat].quantile(q=0.95) for feat in model_independent_data.columns.tolist()]

        #Adding summary fields to save in the dictionary
        model_var_dict['variable_name'] =  model_independent_data.columns.tolist()
        model_var_dict['feat_imp'] = list(model_to_save.feature_importances_)
        model_var_dict['feat_Mean'] = mean_list
        model_var_dict['feat_SD'] = SD_list
        model_var_dict['feat_Min'] = min_list
        model_var_dict['feat_Max'] = max_list
        model_var_dict['feat_5pct'] = list_5_pct
        model_var_dict['feat_10pct'] = list_10_pct
        model_var_dict['feat_90pct'] = list_90_pct
        model_var_dict['feat_95pct'] = list_95_pct
        model_var_dict['bus_day_count_mean'] = model_data_pred['bus_day_count'].mean()
        model_var_dict['bus_day_count_pred_mean'] = model_data_pred['bus_day_count_pred'].mean()
        model_var_dict['bus_day_count_SD'] = model_data_pred['bus_day_count'].std()
        model_var_dict['bus_day_count_pred_SD'] = model_data_pred['bus_day_count_pred'].std()
        model_var_dict['bus_day_count_5_pct'] = model_data_pred['bus_day_count'].quantile(0.05)
        model_var_dict['bus_day_count_pred_5_pct'] = model_data_pred['bus_day_count_pred'].quantile(0.05)
        model_var_dict['bus_day_count_10_pct'] = model_data_pred['bus_day_count'].quantile(0.1)
        model_var_dict['bus_day_count_pred_10_pct'] = model_data_pred['bus_day_count_pred'].quantile(0.1)
        model_var_dict['bus_day_count_90_pct'] = model_data_pred['bus_day_count'].quantile(0.9)
        model_var_dict['bus_day_count_pred_90_pct'] = model_data_pred['bus_day_count_pred'].quantile(0.9)
        model_var_dict['bus_day_count_95_pct'] = model_data_pred['bus_day_count'].quantile(0.95)
        model_var_dict['bus_day_count_pred_95_pct'] = model_data_pred['bus_day_count_pred'].quantile(0.95)
        model_var_dict['bus_day_count_min'] = model_data_pred['bus_day_count'].min()
        model_var_dict['bus_day_count_pred_min'] = model_data_pred['bus_day_count_pred'].min()
        model_var_dict['bus_day_count_max'] = model_data_pred['bus_day_count'].max()
        model_var_dict['bus_day_count_pred_max'] = model_data_pred['bus_day_count_pred'].max()
        model_var_dict['train_mape'] = train_mape
        model_var_dict['test_mape'] = test_mape
        model_var_dict['train_smape'] = train_smape
        model_var_dict['test_smape'] = test_smape
        model_var_dict['train_rmse'] = train_rmse
        model_var_dict['test_rmse'] = test_rmse
        model_var_dict['No_of_datapoints'] = model_independent_data.shape[0]
        model_var_dict['Material_ID'] = mat_id
        summary_df = pd.DataFrame.from_dict(model_var_dict)

      else:
        #Creating independent feature dataset
        model_independent_data = model_data.drop(columns=['Purchase_Order', 'PO_Item', 'PO_Create_Date', 'Material_No.', 
                                                          'Material_Description', 'bus_day_count'])

        #Creating target variable
        model_target = model_data['bus_day_count']

        #Creating train test split
        X_train, X_test, Y_train, Y_test = train_test_split(model_independent_data, model_target, test_size = 0.3, random_state = random_state)

        X_train['bus_day_count'] = Y_train
        X_test['bus_day_count'] = Y_test
        Y_train_pred_df = X_train.groupby(['Plant_ID', 'updated_VS_ID'])['bus_day_count'].mean().reset_index()
        Y_train_pred_df.rename(columns={'bus_day_count':'bus_day_count_pred'}, inplace=True)
        Y_train_pred_df_final = X_train.merge(Y_train_pred_df, on=['Plant_ID', 'updated_VS_ID'], how='left')
        train_mape = 100*mape(Y_train_pred_df_final['bus_day_count'], Y_train_pred_df_final['bus_day_count_pred'])
        train_smape = smape(Y_train_pred_df_final['bus_day_count'], Y_train_pred_df_final['bus_day_count_pred'])
        train_rmse = mse(Y_train_pred_df_final['bus_day_count'], Y_train_pred_df_final['bus_day_count_pred'], squared=False)

        Y_test_pred_df = X_test.groupby(['Plant_ID', 'updated_VS_ID'])['bus_day_count'].mean().reset_index()
        Y_test_pred_df.rename(columns={'bus_day_count':'bus_day_count_pred'}, inplace=True)
        Y_test_pred_df_final = X_test.merge(Y_test_pred_df, on=['Plant_ID', 'updated_VS_ID'], how='left')
        test_mape = 100*mape(Y_test_pred_df_final['bus_day_count'], Y_test_pred_df_final['bus_day_count_pred'])
        test_smape = smape(Y_test_pred_df_final['bus_day_count'], Y_test_pred_df_final['bus_day_count_pred'])
        test_rmse = mse(Y_test_pred_df_final['bus_day_count'], Y_test_pred_df_final['bus_day_count_pred'], squared=False)

        y_pred_df = model_data.groupby(['Plant_ID', 'updated_VS_ID'])['bus_day_count'].mean().reset_index()
        y_pred_df.rename(columns={'bus_day_count':'bus_day_count_pred'}, inplace=True)
        model_data_pred = model_data.merge(y_pred_df, on=['Plant_ID', 'updated_VS_ID'], how='left')
        summary_df = model_data_pred.copy()
        summary_df['train_mape'] = train_mape
        summary_df['test_mape'] = test_mape
        summary_df['train_smape'] = train_smape
        summary_df['test_smape'] = test_smape
        summary_df['train_rmse'] = train_rmse
        summary_df['test_rmse'] = test_rmse
        summary_df['No_of_datapoints'] = model_independent_data.shape[0]
        summary_df['Material_ID'] = mat_id
        summary_df = summary_df[['Material_ID', 'Material_Description', 'train_mape', 'test_mape', 
                                 'train_smape', 'test_smape', 'train_rmse', 'test_rmse', 'No_of_datapoints']].drop_duplicates()

      #creating dataframe for scored data
      model_data_pred = model_data_pred[['Plant_ID', 'updated_VS_ID', 'Material_No.', 'Material_Description', 'Purchase_Order', 'PO_Item',
                                         'PO_Create_Date', 'bus_day_count', 'bus_day_count_pred']]

      #Creating summary dataframe out of summary dictionary and appending to summary_df_list to store
      summary_df_list.append(summary_df)

      #appending predicted data to model_data_pred_df_list to store
      model_data_pred_df_list.append(model_data_pred)
      #Adding Material_No. to outlier data for reference
      outlier_mat_df['Material_No.'] = mat_id

      #Appending model data to model_data_transformed list to store
      model_data_transformed.append(model_data)

      #Appending outlier data to outlier data list to store
      outlier_data_list.append(outlier_mat_df)
    except:
      print('Not able to create model for Material ID: ', mat_id)
      mat_id_not_modelled.append(mat_id)
      
  summary_file_output = 'summ_df_'+model_name+'_'+run_date
  pred_data_output = 'pred_df_'+model_name+'_'+run_date
  outlier_data_output = 'outlier_df_'+model_name+'_'+run_date
  transformed_data_output = 'transformed_df_'+model_name+'_'+run_date
  
  ##Saving outputs
  #Saving summary file
  summary_df_final = pd.concat(summary_df_list, ignore_index=True)
  summary_df_final_table = spark.createDataFrame(summary_df_final)
  summary_df_final_table.write.saveAsTable(summary_file_output, mode = 'overwrite')

  #Saving predicted output
  model_data_pred_df_final = pd.concat(model_data_pred_df_list, ignore_index=True)
  model_data_pred_df_final_table = spark.createDataFrame(model_data_pred_df_final)
  model_data_pred_df_final_table.write.saveAsTable(pred_data_output, mode = 'overwrite')

  #Saving outlier data
  outlier_data_final = pd.concat(outlier_data_list, ignore_index=True)
  outlier_data_final_table = spark.createDataFrame(outlier_data_final)
  outlier_data_final_table.write.saveAsTable(outlier_data_output, mode = 'overwrite')

  #Saving transformed model data
  model_data_transformed_final = pd.concat(model_data_transformed, ignore_index=True)
  model_data_transformed_final_table = spark.createDataFrame(model_data_transformed_final) 
  model_data_transformed_final_table.write.saveAsTable(transformed_data_output, mode = 'overwrite')

In [0]:
#Reading summary files
summ_df_list = []
for model_name in model_list:
  summ_df = spark.read.table('summ_df_'+model_name+'_'+run_date)
  summ_df= summ_df.toPandas()
  summ_df = summ_df.sort_values('Material_ID')[['Material_ID', 'train_mape', 
                                                'test_mape']].drop_duplicates().reset_index().drop(columns=['index'])
  if model_name != 'avg':
    model_name = model_name+'_reg'
  summ_df.rename(columns={'train_mape':'train_mape_'+model_name, 'test_mape':'test_mape_'+model_name}, inplace=True)
  summ_df_list.append(summ_df)

summ_df_final = pd.concat(summ_df_list, axis=1)
summ_df_final = summ_df_final.loc[:,~summ_df_final.columns.duplicated()]

In [0]:
#Readin pred files
pred_df_list = []
for model_name in model_list:
  pred_df = spark.read.table('pred_df_'+model_name+'_'+run_date)
  pred_df= pred_df.toPandas()
  if model_name != 'avg':
    model_name = model_name+'_reg'
  pred_df['model_name'] = model_name
  pred_df_list.append(pred_df)

pred_df_final = pd.concat(pred_df_list, ignore_index=True)

In [0]:
#Function for generating model score to select best model
def score_generation(pred_df, summ_df, mat_id, model_name):
  mat_model_pred_df = pred_df[(pred_df['Material_No.']==mat_id) & (pred_df['model_name']==model_name)]
  mat_model_pred_df['target_decile'] = pd.cut(mat_model_pred_df['bus_day_count'], 10, labels=False)
  mat_model_pred_df['target_pred_decile'] = pd.cut(mat_model_pred_df['bus_day_count_pred'], 10, labels=False)
  decile_1_target_avg = mat_model_pred_df[mat_model_pred_df['target_decile']==0]['bus_day_count'].mean()
  decile_10_target_avg = mat_model_pred_df[mat_model_pred_df['target_decile']==9]['bus_day_count'].mean()
  decile_1_target_pred_avg = mat_model_pred_df[mat_model_pred_df['target_decile']==0]['bus_day_count_pred'].mean()
  decile_10_target_pred_avg = mat_model_pred_df[mat_model_pred_df['target_decile']==9]['bus_day_count_pred'].mean()
  target_spread = abs(decile_10_target_avg-decile_1_target_avg)
  target_pred_spread = abs(decile_10_target_pred_avg-decile_1_target_pred_avg)
  spread_diff = abs(target_pred_spread-target_spread)
  mat_summ_df = summ_df[(summ_df['Material_ID']==mat_id)]
  model_mape = mat_summ_df['test_mape_'+model_name]
  model_score = (model_mape+spread_diff)/2
  return model_score

In [0]:
pred_df_final['model_name'].unique().tolist()

Out[9]: ['xgb_reg', 'rf_reg', 'avg']

In [0]:
#Applying the above function to each material to select best model for each material
mat_ids = pred_df_final['Material_No.'].unique().tolist()
model_list = pred_df_final['model_name'].unique().tolist()
model_score_df_list = []
for mat_id in mat_ids:
  mat_summ_df = summ_df_final[(summ_df_final['Material_ID']==mat_id)]
  for model_name in model_list:
    print(mat_id, model_name)
    try:
      model_score = score_generation(pred_df_final, summ_df_final, mat_id, model_name)
    except:
      model_score = 2000
    mat_summ_df[model_name+'_score'] = model_score
    model_score_df_list.append(mat_summ_df)
model_score_df_final = pd.concat(model_score_df_list, ignore_index=0)

43685562 xgb_reg
43685562 rf_reg
43685562 avg
43521046 xgb_reg
43521046 rf_reg
43521046 avg
43064838 xgb_reg
43064838 rf_reg
43064838 avg
43128357 xgb_reg
43128357 rf_reg
43128357 avg
22002379 xgb_reg
22002379 rf_reg
22002379 avg
22001316 xgb_reg
22001316 rf_reg
22001316 avg
22001170 xgb_reg
22001170 rf_reg
22001170 avg
22000873 xgb_reg
22000873 rf_reg
22000873 avg
22003040 xgb_reg
22003040 rf_reg
22003040 avg
43141133 xgb_reg
43141133 rf_reg
43141133 avg
22002001 xgb_reg
22002001 rf_reg
22002001 avg
22015528 xgb_reg
22015528 rf_reg
22015528 avg
22013995 xgb_reg
22013995 rf_reg
22013995 avg
22014016 xgb_reg
22014016 rf_reg
22014016 avg
43995824 xgb_reg
43995824 rf_reg
43995824 avg
43125651 xgb_reg
43125651 rf_reg
43125651 avg
22015534 xgb_reg
22015534 rf_reg
22015534 avg
22001220 xgb_reg
22001220 rf_reg
22001220 avg
22015533 xgb_reg
22015533 rf_reg
22015533 avg
43854805 xgb_reg
43854805 rf_reg
43854805 avg
43428856 xgb_reg
43428856 rf_reg
43428856 avg
43428850 xgb_reg
43428850 rf_reg
43428850 avg
43433857 xgb_reg
43433857 rf_reg
43433857 avg
22001664 xgb_reg
22001664 rf_reg
22001664 avg
43559239 xgb_reg
43559239 rf_reg
43559239 avg
44082209 xgb_reg
44082209 rf_reg
44082209 avg
44082213 xgb_reg
44082213 rf_reg
44082213 avg
43642252 xgb_reg
43642252 rf_reg
43642252 avg
43646226 xgb_reg
43646226 rf_reg
43646226 avg
43646018 xgb_reg
43646018 rf_reg
43646018 avg
44045819 xgb_reg
44045819 rf_reg
44045819 avg
43941480 xgb_reg
43941480 rf_reg
43941480 avg
22019102 xgb_reg
22019102 rf_reg
22019102 avg
22019193 xgb_reg
22019193 rf_reg
22019193 avg
22019093 xgb_reg
22019093 rf_reg
22019093 avg
22019024 xgb_reg
22019024 rf_reg
22019024 avg
22001250 xgb_reg
22001250 rf_reg
22001250 avg
22012870 xgb_reg
22012870 rf_reg
22012870 avg
22014040 xgb_reg
22014040 rf_reg
22014040 avg
22013676 xgb_reg
22013676 rf_reg
22013676 avg
43049674 xgb_reg
43049674 rf_reg
43049674 avg
22013712 xgb_reg
22013712 rf_reg
22013712 avg
22013710 xgb_reg
22013710 rf_reg
22013710 avg
43684716 xgb_reg
43684716 rf_reg
43684716 avg
43684718 xgb_reg
43684718 rf_reg
43684718 avg
22019126 xgb_reg
22019126 rf_reg
22019126 avg
22002560 xgb_reg
22002560 rf_reg
22002560 avg
22002618 xgb_reg
22002618 rf_reg
22002618 avg
22002573 xgb_reg
22002573 rf_reg
22002573 avg
22002218 xgb_reg
22002218 rf_reg
22002218 avg
22002632 xgb_reg
22002632 rf_reg
22002632 avg
22002194 xgb_reg
22002194 rf_reg
22002194 avg
43398957 xgb_reg
43398957 rf_reg
43398957 avg
22002231 xgb_reg
22002231 rf_reg
22002231 avg
22001851 xgb_reg
22001851 rf_reg
22001851 avg
43060953 xgb_reg
43060953 rf_reg
43060953 avg
22002514 xgb_reg
22002514 rf_reg
22002514 avg
44031292 xgb_reg
44031292 rf_reg
44031292 avg
43851043 xgb_reg
43851043 rf_reg
43851043 avg
22002411 xgb_reg
22002411 rf_reg
22002411 avg
43515628 xgb_reg
43515628 rf_reg
43515628 avg
22002248 xgb_reg
22002248 rf_reg
22002248 avg
22002227 xgb_reg
22002227 rf_reg
22002227 avg
43078612 xgb_reg
43078612 rf_reg
43078612 avg
22002629 xgb_reg
22002629 rf_reg
22002629 avg
43201641 xgb_reg
43201641 rf_reg
43201641 avg
22002545 xgb_reg
22002545 rf_reg
22002545 avg
43058740 xgb_reg
43058740 rf_reg
43058740 avg
22019027 xgb_reg
22019027 rf_reg
22019027 avg
22002525 xgb_reg
22002525 rf_reg
22002525 avg
22006408 xgb_reg
22006408 rf_reg
22006408 avg
43916104 xgb_reg
43916104 rf_reg
43916104 avg
22015532 xgb_reg
22015532 rf_reg
22015532 avg
43171410 xgb_reg
43171410 rf_reg
43171410 avg
43758379 xgb_reg
43758379 rf_reg
43758379 avg
43249377 xgb_reg
43249377 rf_reg
43249377 avg
22002012 xgb_reg
22002012 rf_reg
22002012 avg
43548623 xgb_reg
43548623 rf_reg
43548623 avg
43730619 xgb_reg
43730619 rf_reg
43730619 avg
43058759 xgb_reg
43058759 rf_reg
43058759 avg
43434078 xgb_reg
43434078 rf_reg
43434078 avg
43533466 xgb_reg
43533466 rf_reg
43533466 avg
43549260 xgb_reg
43549260 rf_reg
43549260 avg
22019203 xgb_reg
22019203 rf_reg
22019203 avg
43748707 xgb_reg
43748707 rf_reg
43748707 avg
43242205 xgb_reg
43242205 rf_reg
43242205 avg
22001614 xgb_reg
22001614 rf_reg
22001614 av

In [0]:
#Creating best model column based on the model score generated above
model_score_df_final['best_score_model'] = "best score model is " + model_score_df_final[[col for col in model_score_df_final.columns.tolist() if 'score' in col]].T.idxmin()
model_score_df_final['best_score_model'] = model_score_df_final['best_score_model'].str.split(' ').str[4]
model_score_df_final['best_score_model'] = model_score_df_final['best_score_model'].str.split('_score').str[0]
model_score_df_final = model_score_df_final[['Material_ID', 'best_score_model']].drop_duplicates()

#Saving the best model look up table
table_name = 'material_best_model_vlookup_'+run_date
model_score_df_final_table = spark.createDataFrame(model_score_df_final)
model_score_df_final_table.write.saveAsTable(table_name, mode = 'overwrite')